In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from tqdm import tqdm
import numpy as np
import collections
import nbimporter
import warnings
import gensim
import nltk
import math
import sys
import imp


warnings.filterwarnings("ignore")
import Utilities as utility

Importing Jupyter notebook from Utilities.ipynb


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('../dataset/GoogleNews-vectors-negative300.bin', binary=True) 
model.init_sims(replace=True)

In [17]:
doc_summary_pair = utility.loadObj('test_doc_summary_pair')
combined = []

In [ ]:
for pair in tqdm(doc_summary_pair):
    doc = pair[0]
    sentence_list = doc.split('.')
    included_sentences = []
    clean_sentence_list = []
    for sentence in sentence_list:
        if (len(utility.simpleRemoveStopWords(sentence)) > 3):
            clean_sentence_list.append(utility.simpleRemoveStopWords(sentence))
            included_sentences.append(sentence)

    sentence_distance = np.zeros((len(clean_sentence_list), len(clean_sentence_list)), dtype = float)
    for i in range(len(clean_sentence_list)):
        for j in range(len(clean_sentence_list)):
            dist = model.wmdistance(clean_sentence_list[i], clean_sentence_list[j])
            sentence_distance[i][j] = dist
            sentence_distance[j][i] = dist

    average_sentence_distance = 0
    original_sequence = {}
    for i in range(0,len(sentence_distance)):
        average_sentence_distance = sum(sentence_distance[i])/len(sentence_distance)
        original_sequence[i] = average_sentence_distance

    ordered_dic = collections.OrderedDict(sorted(original_sequence.items(), key=lambda x: x[1]))

    compression_ratio = 40
    num_of_sentences = (compression_ratio*len(original_sequence))//100

    summary = []
    i = 1
    for index in ordered_dic:
        summary.append(included_sentences[index])
        if i == num_of_sentences:
            break
        i += 1

    final_summary = ' '.join(summary)
    human_summary = pair[1]

    human_summary = ' '.join(utility.simpleRemoveStopWords(human_summary))
    summaryBigram = utility.findGrams(final_summary.lower(), 2)
    humanBigram = utility.findGrams(human_summary.lower(), 2)
    count = 0
    for element in humanBigram:
        if element in summaryBigram:
            count+=1
            

    result = count/len(humanBigram)
        
    combined.append(result)

 61%|██████    | 181/298 [09:30<12:58,  6.66s/it] 

In [19]:
sum(combined)/len(combined)

[0.225]

In [ ]:
combined.sort(reverse = True)
combined